In [1]:
import os
import matplotlib.pyplot as plt
import numpy as np
import ecm
import configparser
from openpnm.topotools import plot_connections as pconn
from openpnm.topotools import plot_coordinates as pcoord
from ipywidgets import interact, interactive, fixed, interact_manual
import ipywidgets as widgets
import openpnm as op
%matplotlib inline

In [2]:
wrk = op.Workspace()
wrk.settings['loglevel']=50
save_root = 'C:\\Code\\pybamm_pnm_case1b'
config = configparser.ConfigParser()
config.read(os.path.join(save_root, 'config.txt'))

[]

In [3]:
for sec in config.sections():
    print('='*67)
    print(sec)
    print('='*67)
    for key in config[sec]:
        print('!', key.ljust(30, ' '), '!', config.get(sec, key).ljust(30, ' '), '!')
        print('-'*67)

In [4]:
def plot_topology(net):
    inner = net["pore.inner"]
    outer = net["pore.outer"]
    fig = plt.figure(figsize=(10, 10))
    fig = pconn(net, throats=net.throats("throat.neg_cc"), c="blue", fig=fig)
    fig = pconn(net, throats=net.throats("throat.pos_cc"), c="red", fig=fig)
    fig = pcoord(net, pores=net["pore.neg_cc"], c="blue", fig=fig)
    fig = pcoord(net, pores=net["pore.pos_cc"], c="red", fig=fig)
    fig = pcoord(net, pores=net["pore.neg_tab"], c="blue", s=300, fig=fig)
    fig = pcoord(net, pores=net["pore.pos_tab"], c="red", s=300, fig=fig)
    fig = pcoord(net, pores=inner, c="pink", fig=fig)
    fig = pcoord(net, pores=outer, c="yellow", fig=fig)
    fig = pcoord(net, pores=net.pores('free_stream'), c="green", fig=fig)
    fig = pconn(net, throats=net.throats("throat.free_stream"), c="green",
                fig=fig)
    t_sep = net.throats("spm_resistor")
    if len(t_sep) > 0:
        fig = pconn(
            net, throats=net.throats("spm_resistor"),
            c="k", fig=fig
        )

In [5]:
def update_tabs(project, config):
    net = project.network
    sec = 'GEOMETRY'
    print(net.labels())
    for key in ['pos_tab', 'neg_tab']:
        print(key, 'before', net.pores(key))
        net['pore.'+key] = False
        tab_list = json.loads(config.get(sec, key))
        Ps = net.pores(key.strip('_tab')+'_cc')
        Np = float(len(Ps)-1)
        for i, elem in enumerate(tab_list):
            relative_position = float(elem)
            int_id = int(np.around(relative_position*Np))
            net['pore.'+key.strip('s')][Ps[int_id]] = True
        print(key, 'after', net.pores(key))

In [6]:
def update_plot(p1, p2, p3, p4, p5, n1, n2, n3, n4, n5, net):
    pos_Ps = net.pores('pos_cc')
    neg_Ps = net.pores('neg_cc')
    pos_ints = [p1, p2, p3, p4, p5]
    neg_ints = [n1, n2, n3, n4, n5]
    pos_tabs = pos_Ps[pos_ints]
    neg_tabs = neg_Ps[neg_ints]
    net['pore.pos_tab'] = False
    net['pore.neg_tab'] = False
    net['pore.pos_tab'][pos_tabs] = True
    net['pore.neg_tab'][neg_tabs] = True
    plot_topology(net)
    

In [7]:
project, arc_edges = ecm.make_tomo_net(config)
net = project.network
Npcc = net.num_pores('pos_cc')
Nncc = net.num_pores('neg_cc')
print('Num pos', Npcc, 'Num neg', Nncc)

NoSectionError: No section: 'GEOMETRY'

In [8]:
init_pos = [0 + (4*36*i) for i in range(5)]
init_neg = [(Nncc-1) - (4*36*i) for i in range(5)]

NameError: name 'Nncc' is not defined

In [9]:
interact(update_plot, p1=widgets.IntSlider(min=0, max=Npcc, step=1, value=init_pos[0]),
                      p2=widgets.IntSlider(min=0, max=Npcc, step=1, value=init_pos[1]),
                      p3=widgets.IntSlider(min=0, max=Npcc, step=1, value=init_pos[2]),
                      p4=widgets.IntSlider(min=0, max=Npcc, step=1, value=init_pos[3]),
                      p5=widgets.IntSlider(min=0, max=Npcc, step=1, value=init_pos[4]),
                      n1=widgets.IntSlider(min=0, max=Nncc, step=1, value=init_neg[0]),
                      n2=widgets.IntSlider(min=0, max=Nncc, step=1, value=init_neg[1]),
                      n3=widgets.IntSlider(min=0, max=Nncc, step=1, value=init_neg[2]),
                      n4=widgets.IntSlider(min=0, max=Nncc, step=1, value=init_neg[3]),
                      n5=widgets.IntSlider(min=0, max=Nncc, step=1, value=init_neg[4]),
                      net=fixed(net))

NameError: name 'Npcc' is not defined

In [10]:
neg_cc_lens = net['throat.arc_length'][net['throat.neg_cc']]
cum_len = np.cumsum(neg_cc_lens)
P_neg_tab = np.argmin(cum_len < cum_len[-1]/2)

NameError: name 'net' is not defined

In [11]:
pos_cc_lens = net['throat.arc_length'][net['throat.pos_cc']]
cum_len = np.cumsum(neg_cc_lens)
P_pos_tab = np.argmin(cum_len < cum_len[-1]/2)

NameError: name 'net' is not defined

In [43]:
print(P_neg_tab, P_pos_tab)

445 445


In [37]:
np.argwhere(net['pore.pos_tab'])

array([[ 687],
       [ 831],
       [ 975],
       [1119],
       [1263]], dtype=int64)